## README

**exp1中缩写**

IS: Intended Score

AS：Actual Score

H/M/L：High/Middle/Low {1:low,2:median,3:high}

Ss：Score Satisfaction

Sf：Score Fairness

Es:  Evaluator Satisfaction

Ef:  Evaluator Fairness

**exp2中缩写**

IE: Intended Evaluator

AE: Actual Evaluator

Congruency: IE and AE 的一致性


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import seaborn as sns
from datetime import datetime
# close the warning messages
import warnings
warnings.filterwarnings("ignore")



In [2]:
def calculate_age(birth_date_str):
    birth_date = datetime.strptime(birth_date_str, '%Y-%m-%d')
    current_date = datetime.now()
    age = current_date.year - birth_date.year
    if (current_date.month, current_date.day) < (birth_date.month, birth_date.day):
        age -= 1
    return age

In [3]:
timeLocal = time.localtime(time.time())
exp1File = f"无期望选择_{timeLocal.tm_mon}_{timeLocal.tm_mday}_{timeLocal.tm_hour}h{timeLocal.tm_min}.csv"
exp2File = f"有期望选择_{timeLocal.tm_mon}_{timeLocal.tm_mday}_{timeLocal.tm_hour}h{timeLocal.tm_min}.csv"


rawData = pd.read_excel("data.xlsx") # 这个文件名是数据下载之后改的，对应就行
rawData["expIndex"] = 0
for i in range(len(rawData)):
    if "意愿题" in rawData["随机元素"][i]:
        rawData["expIndex"][i] = 2
    else:
        rawData["expIndex"][i] = 1
# rawData中expIndex为1的提取为exp1结构体，expIndex为2的提取为exp2结构体
exp1 = rawData[rawData['expIndex'] == 1]
exp2 = rawData[rawData['expIndex'] == 2]

exp2.index = range(len(exp2))
exp1.index = range(len(exp1))


In [4]:
# exp2新增"gender(M/F)","Stu/Work"	"IE" "AE"	"score(1)/evaluator(2)"	"AS""H/M/L","AS-IS"	SS,	SF,	Ps,	Pf列，全部赋空值
exp2.dropna(axis=1, how='all', inplace=True)
exp2.rename(columns={'您认为您的答案可以得到几分？': 'IS'}, inplace=True)
exp2['gender(M/F)'] = ''
exp2['Stu/Work'] = ''
exp2['IE'] = ''
exp2['AE'] = ''
exp2['congruency'] = ''
exp2['score(1)/evaluator(2)'] = ''
exp2['I_H/M/L'] = ''
exp2['AS'] = ''
exp2['A_H/M/L'] = ''
exp2['AS-IS'] = ''
exp2['SS'] = ''
exp2['SF'] = ''
exp2['ES'] = ''
exp2['EF'] = ''
exp2['age'] = ''


# 给各列赋值
for i in range(len(exp2)):
    if "AI-AI评分+感知 1" in exp2['随机元素'][i]:
        exp2['IE'][i] = 1
        exp2['AE'][i] = 1
        exp2['congruency'][i] = 1
        exp2['score(1)/evaluator(2)'][i] = 1
    elif "AI-AI评分+感知 2" in exp2['随机元素'][i]:
        exp2['IE'][i] = 1
        exp2['AE'][i] = 1
        exp2['congruency'][i] = 1
        exp2['score(1)/evaluator(2)'][i] = 2
    elif "AI-教师评分+感知 1" in exp2['随机元素'][i]:
        exp2['IE'][i] = 1
        exp2['AE'][i] = 2
        exp2['congruency'][i] = 0
        exp2['score(1)/evaluator(2)'][i] = 1
    elif "AI-教师评分+感知 2" in exp2['随机元素'][i]:
        exp2['IE'][i] = 1
        exp2['AE'][i] = 2
        exp2['congruency'][i] = 0
        exp2['score(1)/evaluator(2)'][i] = 2
    elif "教师-AI评分+感知 1" in exp2['随机元素'][i]:
        exp2['IE'][i] = 2
        exp2['AE'][i] = 1
        exp2['congruency'][i] = 0
        exp2['score(1)/evaluator(2)'][i] = 1
    elif "教师-AI评分+感知 2" in exp2['随机元素'][i]:
        exp2['IE'][i] = 2
        exp2['AE'][i] = 1
        exp2['congruency'][i] = 0
        exp2['score(1)/evaluator(2)'][i] = 2
    elif "教师-教师评分+感知 1" in exp2['随机元素'][i]:
        exp2['IE'][i] = 2
        exp2['AE'][i] = 2
        exp2['congruency'][i] = 1
        exp2['score(1)/evaluator(2)'][i] = 1
    elif "教师-教师评分+感知 2" in exp2['随机元素'][i]:
        exp2['IE'][i] = 2
        exp2['AE'][i] = 2
        exp2['congruency'][i] = 1
        exp2['score(1)/evaluator(2)'][i] = 2
        
# 将列名中包含“满分10分”的列相加到“AS”这一列
exp2['AS'] = exp2.filter(regex='满分10分').sum(axis=1)

# 将列名中包含“评分结果是否公平”的列相加到“SF”这一列
exp2['SF'] = exp2.filter(regex='评分结果是否公平').sum(axis=1)

# 将列名中包含“评分结果是否客观”的列相加到“Ss”这一列
exp2['SS'] = exp2.filter(regex='评分结果是否满意').sum(axis=1)

# 将列名中包含“系统是否满意”或者“教师是否满意”的列相加到“ES”这一列
exp2['ES'] = exp2.filter(regex='系统是否满意|教师是否满意').sum(axis=1)

# 将列名中包含“系统是否满意”或者“教师是否满意”的列相加到“ES”这一列
exp2['EF'] = exp2.filter(regex='系统是否公平|教师是否公平').sum(axis=1)

# AS-IS
exp2['AS-IS'] = exp2['AS'] - exp2['IS']

# 如果“您的性别”一列为“男”，则gender(M/F)一列为1，否则为2
exp2['gender(M/F)'] = exp2['您的性别为'].apply(lambda x: 1 if x == '男' else 2)

# 如果“您的职业”一列为“学生”，则Stu/Work一列为1，否则为2
exp2['Stu/Work'] = exp2['您的职业'].apply(lambda x: 1 if x == '在读学生' else 2)

# 如果AS小于4，H/M/L一列为1，如果AS小于7，H/M/L一列为2，否则为3
exp2['A_H/M/L'] = exp2['AS'].apply(lambda x: 1 if x < 4 else 2 if x < 8 else 3)

exp2['I_H/M/L'] = exp2['IS'].apply(lambda x: 1 if x < 4 else 2 if x < 8 else 3)

# 得到年龄
exp2['age'] = exp2['请选择您的生日'].apply(calculate_age)

exp2.to_csv(exp2File, encoding='GB18030')


In [5]:
# exp1新增gender(M/F)，Stu/Work，AI(1)/teacher(2)，score(1)/evaluator(2)，AS，H/M/L，AS-IS，Ss,	SF,	Ps,	Pf列，全部赋空值
exp1.dropna(axis=1, how='all', inplace=True)
exp1.rename(columns={'您认为您的答案可以得到几分？': 'IS'}, inplace=True)
exp1['gender(M/F)'] = ''
exp1['Stu/Work'] = ''
exp1['AE'] = ''
exp1['score(1)/evaluator(2)'] = ''
exp1['I_H/M/L'] = ''
exp1['AS'] = ''
exp1['A_H/M/L'] = ''
exp1['AS-IS'] = ''
exp1['SS'] = ''
exp1['SF'] = ''
exp1['ES'] = ''
exp1['EF'] = ''


# 给各列赋值
for i in range(len(exp1)):
    if "AI评分+感知 1" in exp1['随机元素'][i]:
        exp1.loc[i, 'AE'] = 1
        exp1.loc[i, 'score(1)/evaluator(2)'] = 1
    elif "AI评分+感知 2" in exp1['随机元素'][i]:
        exp1.loc[i, 'AE'] = 1
        exp1.loc[i, 'score(1)/evaluator(2)'] = 2
    elif "教师评分+感知 1" in exp1['随机元素'][i]:
        exp1.loc[i, 'AE'] = 2
        exp1.loc[i, 'score(1)/evaluator(2)'] = 1
    elif "教师评分+感知 2" in exp1['随机元素'][i]:
        exp1.loc[i, 'AE'] = 2
        exp1.loc[i, 'score(1)/evaluator(2)'] = 2
    
        
# 将列名中包含“满分10分”的列相加到“AS”这一列
exp1['AS'] = exp1.filter(regex='满分10分').sum(axis=1)

# 将列名中包含“评分结果是否公平”的列相加到“SF”这一列
exp1['SF'] = exp1.filter(regex='评分结果是否公平').sum(axis=1)

# 将列名中包含“评分结果是否客观”的列相加到“Ss”这一列
exp1['SS'] = exp1.filter(regex='评分结果是否满意').sum(axis=1)

# 将列名中包含“系统是否满意”或者“教师是否满意”的列相加到“ES”这一列
exp1['ES'] = exp1.filter(regex='系统是否满意|教师是否满意').sum(axis=1)

# 将列名中包含“系统是否满意”或者“教师是否满意”的列相加到“ES”这一列
exp1['EF'] = exp1.filter(regex='系统是否公平|教师是否公平').sum(axis=1)

# AS-IS
exp1['AS-IS'] = exp1['AS'] - exp1['IS']

# 如果“您的性别”一列为“男”，则gender(M/F)一列为1，否则为2
exp1['gender(M/F)'] = exp1['您的性别为'].apply(lambda x: 1 if x == '男' else 2)

# 如果“您的职业”一列为“学生”，则Stu/Work一列为1，否则为2
exp1['Stu/Work'] = exp1['您的职业'].apply(lambda x: 1 if x == '在读学生' else 2)

# 如果AS小于4，H/M/L一列为1，如果AS小于7，H/M/L一列为2，否则为3
exp1['A_H/M/L'] = exp1['AS'].apply(lambda x: 1 if x < 4 else 2 if x < 8 else 3)

exp1['I_H/M/L'] = exp1['IS'].apply(lambda x: 1 if x < 4 else 2 if x < 8 else 3)

exp1['age'] = exp1['请选择您的生日'].apply(calculate_age)

exp1 = exp1[exp1['AS-IS']<7]

exp1.to_csv(exp1File, encoding='GB18030')

